
# Performance And Costing Analysis of Ergo Node

## Introduction
This notebook uses `Metrics.sqlite` database collected during full node syncronization.
See `README.md` for detail of database schema.



## Setup
If necessary uncomment and run the following commands to setup necessary packages.


In [1]:
#!pip install ipython-sql
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
!jupyter nbextension enable python-markdown / main


Please specify one nbextension/package at a time


In [2]:
# open sqlite connection to perform queries
import pandas as pd
import sqlite3
conn = sqlite3.connect("../Metrics.sqlite")


## v5.0 Validation
- make sure validateTxStateful recorded for each transaction of the block (should be empty)

In [3]:
pd.read_sql_query(f"""
    -- invalid tx_num
    select b.blockId, t.tx_count as c, b.tx_num as n
    from (select blockId, count(*) as tx_count
          from validateTxStateful
          group by blockId) as t
             join applyTransactions as b on b.blockId = t.blockId
    where c != n;
""", conn)

,blockId,c,n


- make sure recored block cost = sum of recorded tx costs (should be empty)

In [4]:
pd.read_sql_query(f"""
    -- invalid tx cost
    select b.blockId, t.sum_costs as sum_tx_costs, b.cost as block_cost
    from (select blockId, sum(cost) as sum_costs
          from validateTxStateful
          group by blockId) as t
             join applyTransactions as b on b.blockId = t.blockId
    where sum_tx_costs != block_cost;
""", conn)


,blockId,sum_tx_costs,block_cost


- now many transactions have negative cost (should be empty)

In [5]:
pd.read_sql_query(f"""
-- count tx with negative cost
select count(*)
from validateTxStateful
where cost < 0;
""", conn)


,count(*)
0,0


## v5.0 Analysis
### Block Validation Time Analysis
In this section we compare script validation time against block validation time.

#### Sizes of the recorded tables

In [6]:
pd.read_sql_query(f"""
    select * from
    (select count(*) as appendFullBlock
    from appendFullBlock),
    (select count(*) as applyTransactions
    from applyTransactions),
    (select count(*) as createUtxoState
    from createUtxoState),
    (select count(*) as validateTxStateful
    from validateTxStateful),
    (select count(*) as verifyScript
    from verifyScript)
""", conn)


,appendFullBlock,applyTransactions,createUtxoState,validateTxStateful,verifyScript
0,488100,488100,488100,1321800,3823200


#### Comparing Stages of Block Validation
First we look at how much time of the total block validation is spent in applyTransactions.
We group and count blocks by (total time / `applyTransactions` time) ratio.
We see that `applyTransactions` is < 50% for roughly 80% of the blocks.
For more ~40% of the blocks:
1) the ratio is above 3, which means script validation is < 33%
2) further analysis shows that time is spent in creating UtxoState (after applyTransaction)
**Thus, creating UtxoState after application of transactions requires profiling and optimization.**

In [7]:
pd.read_sql_query(f"""
select time_us / t2_us as time_ratio, count(*) as block_count
from (select b1.blockId,
             b1.height,
             b1.tx_num,
             b2.cost,
             b1.time / 1000                       as t1_us,
             b2.time / 1000                       as t2_us,
             b3.time / 1000                       as t3_us,
             (b1.time + b2.time + b3.time) / 1000 as time_us
      from appendFullBlock as b1
               join applyTransactions as b2 on b1.blockId = b2.blockId
               join createUtxoState b3 on b1.blockId = b3.blockId)
group by time_ratio
order by time_ratio;
""", conn)


,time_ratio,block_count
0,1,70293
1,2,155027
2,3,207666
3,4,21667
4,5,20585
5,6,5678
6,7,2472
7,8,1444
8,9,1020
9,10,666


#### Comparing applyTransaction of block with validateTxStateful
Here we further drill down to applyTransactions part of block validation.
Specifically, for each block we compare the time of `UtxoState.applyTransactions` with total time
of `ErgoTransaction.validateStateful` taken for all transactions in the block.
The blocks are grouped by the ratio between times.
We can see that for > 70% blocks the ration is above 2, which suggests that
**`UtxoState.applyTransactions` method need optimizations.**

In [8]:
pd.read_sql_query(f"""
select t.time_ratio / 10 as time_ratio,
       count(*) as block_count,
       round(avg(t.block_time_us), 0)  as avg_block_time_us,
       round(avg(t.tx_time_us), 0) as avg_tx_time_us,
       round(avg(t.tx_count), 1)   as avg_tx_count
from (select b.blockId,
             tx.tx_count,
             b.time / 1000                        as block_time_us,
             tx.sum_tx_time / 1000            as tx_time_us,
             (b.time - tx.sum_tx_time) / 1000 as time_diff_us,
             b.time * 10 / tx.sum_tx_time     as time_ratio
      from (select blockId,
                   sum(time) as sum_tx_time,
                   count(*)  as tx_count
            from validateTxStateful
            group by blockId) as tx
               join applyTransactions as b on b.blockId = tx.blockId) as t
group by t.time_ratio / 10
order by t.time_ratio / 10;
""", conn)


,time_ratio,block_count,avg_block_time_us,avg_tx_time_us,avg_tx_count
0,1,148957,17974.0,15156.0,6.6
1,2,170380,1193.0,425.0,1.0
2,3,167950,1151.0,375.0,1.0
3,4,256,3337.0,749.0,1.3
4,5,138,3965.0,725.0,1.3
...,...,...,...,...,...
63,75,1,31507.0,417.0,1.0
64,82,1,32388.0,392.0,1.0
65,83,1,33706.0,404.0,1.0
66,86,1,41055.0,477.0,1.0


#### Tx/Script validation time ratio
Further down to validation call stack, we observe how much `validateStateful` time larger than
script verification time.
The ratio is computed for each transaction and then the transactions are grouped by integer ratio.
Comparing times in `verifyScript` and `validateTxStateful` metrics.
We see that for most transactions, `verifyScript` is > 50% of `validateStateful`.

In [9]:
pd.read_sql_query(f"""
select t.time_ratio / 10 as time_ratio,
       count(*) as tx_count,
       round(avg(t.tx_time_us), 1) as avg_tx_time_us,
       round(avg(t.script_time_us), 1) as avg_script_time_us,
       round(avg(t.script_count), 1) as avg_script_count
from (select tx.blockId,
             tx.txId,
             t.script_count,
             tx.time / 1000                   as tx_time_us,
             t.sum_script_time / 1000         as script_time_us,
             tx.time * 10 / t.sum_script_time as time_ratio
      from (select blockId,
                   txId,
                   sum(time) as sum_script_time,
                   count(*)  as script_count
            from verifyScript
            group by blockId, txId) as t
               join validateTxStateful as tx on tx.blockId = t.blockId and tx.txId = t.txId) as t
group by t.time_ratio / 10
order by t.time_ratio / 10;
""", conn)


,time_ratio,tx_count,avg_tx_time_us,avg_script_time_us,avg_script_count
0,1,1321331,1806.3,1765.3,2.9
1,2,154,7181.1,2900.2,6.1
2,3,63,11364.8,3407.0,8.5
3,4,43,14648.4,3341.8,7.6
4,5,24,12482.9,2229.4,4.5
5,6,30,11152.2,1712.3,4.7
6,7,24,22004.2,3038.2,6.2
7,8,14,9936.7,1192.9,3.0
8,9,7,18300.3,1928.6,5.1
9,10,10,10140.6,972.2,4.3


We need to further drill down inside the mose populated group.
We build a detailed grouping of the transactions where the ratio <= 1.9,
comparing times in `verifyScript` and `validateTxStateful` metrics.

In [10]:
pd.read_sql_query(f"""
select round(t.time_ratio * 0.1, 1) as time_ratio,
       count(*) as tx_count,
       round(avg(t.tx_time_us), 1) as avg_tx_time_us,
       round(avg(t.script_time_us), 1) as avg_script_time_us,
       round(avg(t.script_count), 1) as avg_script_count
from (select tx.blockId,
             tx.txId,
             t.script_count,
             tx.time / 1000                   as tx_time_us,
             t.sum_script_time / 1000         as script_time_us,
             tx.time * 10 / t.sum_script_time as time_ratio
      from (select blockId,
                   txId,
                   sum(time) as sum_script_time,
                   count(*)  as script_count
            from verifyScript
            group by blockId, txId) as t
               join validateTxStateful as tx on tx.blockId = t.blockId and tx.txId = t.txId) as t
where t.time_ratio <= 19
group by t.time_ratio
order by t.time_ratio;
""", conn)


,time_ratio,tx_count,avg_tx_time_us,avg_script_time_us,avg_script_count
0,1.0,1211507,1934.7,1895.0,3.0
1,1.1,101236,337.2,299.3,1.2
2,1.2,6944,736.0,599.2,1.6
3,1.3,498,2257.4,1673.0,4.4
4,1.4,869,970.4,671.6,2.2
5,1.5,102,4768.1,3073.3,6.2
6,1.6,51,5595.5,3370.3,9.6
7,1.7,44,8933.2,5113.4,10.8
8,1.8,42,6566.0,3518.1,8.1
9,1.9,38,4534.5,2315.4,3.6


Count transactions where script validation <= 80% of `validateStatefull`.
This DOESN'T show big potential for optimizing `validateStateful` outside script evaluation.
Comparing times in `verifyScript` and `validateTxStateful` metrics.

In [11]:
pd.read_sql_query(f"""
select count(*) as tx_count
from (select tx.blockId,
             tx.txId,
             t.script_count,
             tx.time / 1000                   as tx_time_us,
             t.sum_script_time / 1000         as script_time_us,
             tx.time * 10 / t.sum_script_time as time_ratio
      from (select blockId,
                   txId,
                   sum(time) as sum_script_time,
                   count(*)  as script_count
            from verifyScript
            group by blockId, txId) as t
               join validateTxStateful as tx on tx.blockId = t.blockId and tx.txId = t.txId) as t
where t.time_ratio >= 12;
""", conn)


,tx_count
0,9049


Conclusions:
- Creating UtxoState after application of transactions requires profiling and optimization.
- `UtxoState.applyTransactions` method need optimizations.
- In most of the cases the time to validate transaction (`validateStateful` method) is dominated by
the script validation time
- There is a few transactions where script validation not greater than 80% of tx validation time.
- The results suggest that `validateStateful` doesn't require optimizations, or at least it can
be done after the other parts of block validation had been optimized.

### Block Validation Cost Analysis
In this section we compare script validation costs against transaction and block validation costs.

## v5.0 vs v4.0 Cross Analysis

### Cross Version validation
- Make sure recorded data for v5 and v4 have the same `height` and `tx_num` for all blockIds


In [12]:
checks = dict(
    appendFullBlockOk=pd.read_sql_query(f"""
        -- validate appendFullBlock tables
        select *
        from appendFullBlock a1
                 join appendFullBlock4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0,
    applyTransactionsOk=pd.read_sql_query(f"""
        select *
        from applyTransactions a1
                 join applyTransactions4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0,
    createUtxoStateOk=pd.read_sql_query(f"""
        select *
        from createUtxoState a1
                 join createUtxoState4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0
)
if not (checks.get("appendFullBlockOk") and checks.get("applyTransactionsOk") and checks.get(
        "createUtxoStateOk")):
    ok = checks
else:
    ok = "ok"
print(checks)


{'appendFullBlockOk': True, 'applyTransactionsOk': True, 'createUtxoStateOk': True}


### Block Validation Times
First count recorded block validations of v4 and v5 and how much of them can be
joined. This is also validation check, because `common_rows` should be equal to the minimal count.

In [13]:
pd.read_sql_query(f"""
select *, count_rows5 - count_rows4 as v5_v4_rows_diff
from (select count(*) as count_rows5 from applyTransactions),
     (select count(*) as count_rows4 from applyTransactions4),
     (select count(*) as common_rows
      from applyTransactions as t5 join applyTransactions4 t4 on t5.blockId = t4.blockId);
""", conn)


,count_rows5,count_rows4,common_rows,v5_v4_rows_diff
0,488100,473000,473000,15100


The new v5 script interpreter is expected to perform faster comparing to v4.
The following shows the distribution of the blocks across ranges of speedup. We see that for the
most of the blocks the speedup is in a range from 1 (no speedup) to 2 (twice faster).

In [14]:
pd.read_sql_query(f"""
select t4.time / t5.time as speedup, count(*) as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
group by speedup order by speedup;
""", conn)

,speedup,num_blocks
0,0,8048
1,1,449950
2,2,9866
3,3,1950
4,4,962
5,5,734
6,6,547
7,7,206
8,8,191
9,9,132


We see outliers, i.e. blocks which executed slower in v5 than in v4. This can be attributed to
measurement fluctuations. Such blocks will be simply filtered out in the following analysis.

We also see a long tail of blocks with higher than 2 speedups.

In [15]:
pd.read_sql_query(f"""
select sum(num_blocks) from (
select t4.time / t5.time as speedup, count(*) as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
where speedup >= 2
group by speedup order by speedup
)
""", conn)


,sum(num_blocks)
0,15002


Let's see how the speedup is spread over ranges of the blockchain.
The block `b` falls in `range` if `range = b.height / 100000`. The blocks of each range are counted,
and the average speedup over the range is computed.

In [16]:
pd.read_sql_query(f"""
select t5.height / 100000                            as range,
       round(avg(t4.time * 100 / t5.time * 0.01), 2) as avg_speedup,
       count(*)                                      as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
where t4.time / t5.time >= 1
group by range order by range;
""", conn)


,range,avg_speedup,num_blocks
0,0,1.26,97603
1,1,1.29,98368
2,2,1.34,99011
3,3,1.40,98192
4,4,1.45,71778


We observe that the average speedup increase towards the most recent range, which is expected, because
more and more complex contracts are used on chain over time. The v5 script interpreter is by design
faster on complex contracts.

If we further constrain speedup to be above 1.2, we get the following distribution of blocks.

In [17]:
pd.read_sql_query(f"""
select t5.height / 100000                            as range,
       round(avg(t4.time * 100 / t5.time * 0.01), 2) as avg_speedup,
       count(*) as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
where t4.time * 100 / t5.time * 0.01 >= 1.2
group by range order by range
""", conn)

,range,avg_speedup,num_blocks
0,0,1.34,60383
1,1,1.35,68973
2,2,1.39,79345
3,3,1.44,84885
4,4,1.52,60029


Note, this speedups are due to improvements in script evaluation which is further analyzed in the
[next section](#script-validation-time). However, as shown in [Comparing Stages of Block
Validation](#comparing-stages-of-block-validation), for most of the blocks, script evaluation is
less than 50% of the block validation time, so the impact of script speedups is limited by this
factor.


### Script Validation Time
In this section we focus on script evaluation part only (i.e. Interpreter.verify method), which is
measured and recorded in `verifyScript` table.

First count recorded script validations of v4 and v5 and how much of them can be
joined. This is also validation check, because `common_rows` should be equal to the minimal count.

In [18]:
pd.read_sql_query(f"""
select *, total_rows5 - total_rows4 as v5_v4_rows_diff from
(select count(*) as total_rows5
from verifyScript as s),
(select count(*) as total_rows4
from verifyScript4 as s),
(select count(*) as common_rows
from verifyScript as t5
         join verifyScript4 t4
              on t5.blockId = t4.blockId
                  and t5.txId = t4.txId
                  and t5.boxIndex = t4.boxIndex);
""", conn)


,total_rows5,total_rows4,common_rows,v5_v4_rows_diff
0,3823200,3537350,3537350,285850


  For common recorded script we compute the total time spend in script validation for both v4 and v5
  and compare them. Then compute the expected percent of total script time reduction after switching
  to v5.0.

In [19]:
pd.read_sql_query(f"""
select times.total_time4 / 1000                                   as total_time4_us,
       times.total_time5 / 1000                                   as total_time5_us,
       (times.total_time4 - times.total_time5) / 1000             as total_diff_us,
       round((1 - round(times.total_time5 * 100 / times.total_time4 * 0.01, 1)) * 100, 1) as percent_of_reduction
from (select sum(t5.time) as total_time5,
             sum(t4.time) as total_time4
      from verifyScript as t5
               join verifyScript4 t4
                    on t5.blockId = t4.blockId
                        and t5.txId = t4.txId
                        and t5.boxIndex = t4.boxIndex) as times;
""", conn)

,total_time4_us,total_time5_us,total_diff_us,percent_of_reduction
0,3175910119,2207370816,968539302,30.0
